In [2]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuraciones
sns.set(font_scale=1.2)

In [3]:
# Ruta al archivo CSV
file_path = '../data/raw/Summary.csv'

# Cargar el dataset con el separador correcto
df = pd.read_csv(file_path, sep=';')  # Usar punto y coma como separador

# Mostrar las primeras filas
print(f"Shape del dataset: {df.shape}")
df.head()

Shape del dataset: (12767, 4)


,Date,Flower,Color,Units Selection
0,01/10/2024 0:00,Alstroemeria,Burgundy,0
1,01/10/2024 0:00,Alstroemeria,DarkPink,11200
2,01/10/2024 0:00,Alstroemeria,Green,0
3,01/10/2024 0:00,Alstroemeria,Lavender,2400
4,01/10/2024 0:00,Alstroemeria,Orange,6000


In [4]:
# Información general del dataset
print("Información del dataset:")
df.info()

# Estadísticas descriptivas de la columnas 'Flower' y 'Color'
print("\nEstadísticas descriptivas de la columnas 'Flower' y 'Color':")
df[['Flower', 'Color']].describe()

# Estadísticas descriptivas de la columna 'Units Selection'
print("\nEstadísticas descriptivas de la columna 'Units Selection':")
df['Units Selection'].describe()


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12767 entries, 0 to 12766
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             12767 non-null  object
 1   Flower           12767 non-null  object
 2   Color            12767 non-null  object
 3   Units Selection  12767 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 399.1+ KB

Estadísticas descriptivas de la columnas 'Flower' y 'Color':

Estadísticas descriptivas de la columna 'Units Selection':


count    12767.000000
mean      3710.346048
std       6028.213003
min          0.000000
25%        400.000000
50%       1700.000000
75%       4580.000000
max      82512.000000
Name: Units Selection, dtype: float64

In [5]:
# Valores únicos por columna categórica
for col in ['Flower', 'Color']:
    print(f"\n{col} - {df[col].nunique()} valores únicos:")
    print(df[col].value_counts().head(10))


Flower - 77 valores únicos:
Flower
Daisy            1087
Novelty           953
Cushion           916
Carnation         891
Cremon            737
Minicarnation     728
Alstroemeria      614
Rose - Color      544
Mum Ball          500
Rose - Spray      495
Name: count, dtype: int64

Color - 60 valores únicos:
Color
White       1338
Green       1221
Yellow      1146
Purple      1091
Pink         970
Lavender     782
Red          709
DarkPink     680
Orange       448
Peach        414
Name: count, dtype: int64


In [6]:
# Crear una copia del dataframe original
df_clean = df.copy()

# Paso 1: Eliminar filas duplicadas
duplicados_antes = df_clean.shape[0]
df_clean = df_clean.drop_duplicates()
duplicados_eliminados = duplicados_antes - df_clean.shape[0]
print(f"Se eliminaron {duplicados_eliminados} filas duplicadas.")

# Paso 2: Verificar valores nulos
null_counts = df_clean.isnull().sum()
print("\nConteo de valores nulos por columna:")
print(null_counts[null_counts > 0])

Se eliminaron 0 filas duplicadas.

Conteo de valores nulos por columna:
Series([], dtype: int64)


In [7]:
# Convertir columna de fecha con formato específico
if 'Date' in df_clean.columns:
    # Mostrar primeros valores de fecha para diagnóstico
    print("Primeros 5 valores de fecha:")
    for i in range(5):
        print(f"  {i+1}: {df_clean['Date'].iloc[i]}")
    
    # Convertir fechas con formato específico
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%d/%m/%Y %H:%M', errors='coerce')
    
    # Verificar que las fechas estén en el rango esperado
    min_date = df_clean['Date'].min()
    max_date = df_clean['Date'].max()
    print(f"Rango de fechas: {min_date} a {max_date}")
    
    # Si las fechas están en 2024, convertirlas a 2022
    if max_date.year > 2023:
        print("Ajustando años para que estén en el rango actual...")
        # Restar 2 años a todas las fechas
        df_clean['Date'] = df_clean['Date'] - pd.DateOffset(years=2)
        min_date = df_clean['Date'].min()
        max_date = df_clean['Date'].max()
        print(f"Nuevo rango de fechas: {min_date} a {max_date}")

# Convertir Units Selection a numérico
if 'Units Selection' in df_clean.columns:
    df_clean['Units Selection'] = pd.to_numeric(df_clean['Units Selection'], errors='coerce')
    print(f"Columna Units Selection convertida a tipo numérico.")

Primeros 5 valores de fecha:
  1: 01/10/2024 0:00
  2: 01/10/2024 0:00
  3: 01/10/2024 0:00
  4: 01/10/2024 0:00
  5: 01/10/2024 0:00
Rango de fechas: 2024-10-01 00:00:00 a 2024-12-31 00:00:00
Ajustando años para que estén en el rango actual...
Nuevo rango de fechas: 2022-10-01 00:00:00 a 2022-12-31 00:00:00
Columna Units Selection convertida a tipo numérico.


In [8]:
# Cambiar el nombre de la columna 'Units Selection' a 'Stems'
df_clean.rename(columns={'Units Selection': 'Stems'}, inplace=True)

# Verificar que la columna 'Stems' se haya creado correctamente
if 'Stems' in df_clean.columns:
    print("Columna 'Stems' creada correctamente.")

Columna 'Stems' creada correctamente.


In [9]:
# Cambiar el nombre de la columna date a 'DueDate'
df_clean.rename(columns={'Date': 'DueDate'}, inplace=True)

# Verificar que la columna 'DueDate' se haya creado correctamente
if 'DueDate' in df_clean.columns:
    print("Columna 'DueDate' creada correctamente.")

Columna 'DueDate' creada correctamente.


In [10]:
# Agregar columnas de año, mes y día para análisis
df_clean['Year'] = df_clean['DueDate'].dt.year
df_clean['Month'] = df_clean['DueDate'].dt.month
df_clean['Day'] = df_clean['DueDate'].dt.day

# Verificar que las columnas de fecha se hayan creado correctamente
if all(col in df_clean.columns for col in ['Year', 'Month', 'Day']):
    print("Columnas de fecha creadas correctamente.")

# Eliminar la columna 'DueDate'
df_clean.drop(columns=['DueDate'], inplace=True)

Columnas de fecha creadas correctamente.


In [11]:
# Convertir las columnas de fecha añoa, mes y día a tipo datetime
df_clean['DueDate'] = pd.to_datetime(df_clean[['Year', 'Month', 'Day']])

# Verificar que las columnas de fecha se hayan creado correctamente
if all(col in df_clean.columns for col in ['DueDate']):
    print("Columnas de fecha creadas correctamente.")

Columnas de fecha creadas correctamente.


In [12]:
# Mostrar las primeras filas del dataframe
print(df_clean.head())

# Mostrar las últimas filas del dataframe
print(df_clean.tail())

# Mostrar información general del dataframe
print(df_clean.info())

# Mostrar estadísticas descriptivas del dataframe
print(df_clean.describe())


         Flower     Color  Stems  Year  Month  Day    DueDate
0  Alstroemeria  Burgundy      0  2022     10    1 2022-10-01
1  Alstroemeria  DarkPink  11200  2022     10    1 2022-10-01
2  Alstroemeria     Green      0  2022     10    1 2022-10-01
3  Alstroemeria  Lavender   2400  2022     10    1 2022-10-01
4  Alstroemeria    Orange   6000  2022     10    1 2022-10-01
           Flower   Color  Stems  Year  Month  Day    DueDate
12762  Super Mums  Yellow   1250  2022     12   31 2022-12-31
12763    Treefern   Green    180  2022     12   31 2022-12-31
12764    Veronica    Pink   4350  2022     12   31 2022-12-31
12765    Veronica  Purple   5550  2022     12   31 2022-12-31
12766    Veronica   White    150  2022     12   31 2022-12-31
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12767 entries, 0 to 12766
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Flower   12767 non-null  object        
 1   Color   

In [ ]:
# Guardar el dataset limpio
output_path = '../data/processed/summary_clean.csv'
df_clean.to_csv(output_path, index=False)
print(f"\nDataset limpio guardado en: {output_path}") 


Dataset limpio guardado en: ../data/processed/summary_clean.csv
